## xckd-scraper.py

Le but de ce notebook est de montrer un script de récupération de données provenant d'une source tierce (ici la liste des épisodes du webcomic XKCD), et leur transformation dans un format reconnu par l'un des outils d'imports de Wikidata (en l'occurrence Quick Statements.)

![XKCD - Manifestant wikipédien](https://upload.wikimedia.org/wikipedia/commons/7/77/Ref_necessaire.png)

In [8]:
import requests  # Librairie pour faire une requête web
from bs4 import BeautifulSoup  # Librairie pour interpréter une page web
from SPARQLWrapper import SPARQLWrapper, JSON  # Librairie pour faire une requête SPARQL

# Liste des langues pour lesquelles on va générer les libellés
languages = ['fr', 'en', 'br', 'de']

"""
Fonction pour faire une requête SPARQL sur le point d'accès de Wikidata
et renvoyer le résultat sous la forme d'un dictionnaire Python
"""
def wikidata_sparql_query(query):
    endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)

    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results

"""
Fonction qui renvoie la valeure ordinale en anglais d'un nombre positif ou nul
(par exemple, 1 → 1st, 2 → 2nd, 3 → 3rd, 4 → 4th)
Utilisé pour la description en anglais
"""
def ordinal(value):
    """
    Converts zero or a *positive* integer (or its string
    representation) to an ordinal value.
    """
    try:
        value = int(value)
    except ValueError:
        return value

    if value % 100 // 10 != 1:
        if value % 10 == 1:
            ordval = u"%d%s" % (value, "st")
        elif value % 10 == 2:
            ordval = u"%d%s" % (value, "nd")
        elif value % 10 == 3:
            ordval = u"%d%s" % (value, "rd")
        else:
            ordval = u"%d%s" % (value, "th")
    else:
        ordval = u"%d%s" % (value, "th")

    return ordval

"""
Fonction qui renvoie une déclaration avec sa propriété, sa valeur et
sa source au format attendu par QuickStatements
"""
def statement(prop, value, source):
    return "LAST\t{}\t{}\tS854\t\"{}\"".format(prop, value, source)


"""
Première étape : récupérer la liste des épisodes déjà présents sur Wikidata, pour
ne pas les importer à nouveau. On le fait par le biais d'une requête SPARQL
"""

imported_episodes = []

results = wikidata_sparql_query("""
SELECT DISTINCT ?episode ?episodeLabel ?number WHERE {
  ?episode wdt:P31 wd:Q838795 .
  ?episode wdt:P361 wd:Q13915 .
  ?episode wdt:P433 ?number .

  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
  }
} ORDER BY xsd:integer(?number)
""")

for r in results["results"]["bindings"]:
    try:
        imported_episodes.append(int(r["number"]["value"]))
    except:
        print("{} has no episode number.".format(r))

# On récupère le numéro d'épisode le plus élevé présent sur Wikidata
latest_imported_episode = max(imported_episodes)

# On vérifie qu'il ne manque pas d'épisodes avant celui-là...
for i in range(1, latest_imported_episode):
    # Sachant qu'il n'y a pas d'épisode 404.
    if i not in imported_episodes and i != 404:
        print('Episode {} is missing :('.format(i))


# On récupère la liste des épisodes sur le site xkcd
root_url = 'http://www.xkcd.com'
index_url = root_url + '/archive/index.html'

response = requests.get(index_url)
soup = BeautifulSoup(response.text, "lxml")

episodes = soup.select('div#middleContainer a')
episodes.reverse()

for e in episodes:
    # Récupération des données de base
    title = "\"" + e.get_text() + "\"" or ""
    urlbit = e.attrs.get('href') or ""
    episode_url = root_url + urlbit
    episodenumber = urlbit.replace("/", "")
    
    # Si l'épisode n'existe pas , on le crée
    if int(episodenumber) > latest_imported_episode:

        date = "+0000000" + '-'.join([
            "{0:0>2}".format(v) for v in e.attrs.get('title').split("-")]) + \
            "T00:00:00Z/11" or ""

        print("CREATE")
        for l in languages:
            print("LAST\tL{}\t{}".format(l, title))
            print("LAST\tA{}\t\"xkcd {}\"".format(l, episodenumber))

        print("LAST\tDfr\t\"strip de xkcd n°{}\"".format(episodenumber))
        print("LAST\tDde\t\"Folge des Webcomics xkcd\"")
        print("LAST\tDen\t\"{} strip of the webcomic xkcd\"".format(
            ordinal(episodenumber)))

        print(statement("P31", "Q838795", episode_url))  # instance of
        print(statement("P361", "Q13915", episode_url))  # part of        
        print(statement("P433", '"' + episodenumber + '"', episode_url))  # nb
        print(statement("P577", date, episode_url))  # date
        print(statement("P50", "Q285048", episode_url))  # Author: R. Munroe
        print(statement("P2699", '"' + episode_url + '"', episode_url))  # URL
        print(statement("P364", "Q1860", episode_url))  # Language: English
        print(statement("P275", "Q6936496", episode_url))  # Licence: CC-BY-NC
        print("")


CREATE
LAST	Lfr	"Thumb War"
LAST	Afr	"xkcd 1753"
LAST	Len	"Thumb War"
LAST	Aen	"xkcd 1753"
LAST	Lbr	"Thumb War"
LAST	Abr	"xkcd 1753"
LAST	Lde	"Thumb War"
LAST	Ade	"xkcd 1753"
LAST	Dfr	"strip de xkcd n°1753"
LAST	Dde	"Folge des Webcomics xkcd"
LAST	Den	"1753rd strip of the webcomic xkcd"
LAST	P31	Q838795	S854	"http://www.xkcd.com/1753/"
LAST	P361	Q13915	S854	"http://www.xkcd.com/1753/"
LAST	P433	"1753"	S854	"http://www.xkcd.com/1753/"
LAST	P577	+00000002016-10-31T00:00:00Z/11	S854	"http://www.xkcd.com/1753/"
LAST	P50	Q285048	S854	"http://www.xkcd.com/1753/"
LAST	P2699	"http://www.xkcd.com/1753/"	S854	"http://www.xkcd.com/1753/"
LAST	P364	Q1860	S854	"http://www.xkcd.com/1753/"
LAST	P275	Q6936496	S854	"http://www.xkcd.com/1753/"

CREATE
LAST	Lfr	"Tornado Safety Tips"
LAST	Afr	"xkcd 1754"
LAST	Len	"Tornado Safety Tips"
LAST	Aen	"xkcd 1754"
LAST	Lbr	"Tornado Safety Tips"
LAST	Abr	"xkcd 1754"
LAST	Lde	"Tornado Safety Tips"
LAST	Ade	"xkcd 1754"
LAST	Dfr	"strip de xkcd n°1754"
LAST	Dde	"Folge 

Il ne reste plus qu'à copier-coller le résultat dans l'outil [QuickStatements](https://tools.wmflabs.org/quickstatements/#) ! Note : l'outil est en phase de refonte importante, l'interface est donc en travaux et sera améliorée.

À la première connexion à l'outil, il faut donner l'autorisation à QuickStatements d'utiliser votre compte Wikidata, en cliquant sur le lien « Log in » en haut à droite.

Ensuite, il faut cliquer sur le menu « Import Commands », puis « Version 1 format ». Cela ouvre un formulaire dans lequel on peut coller le résultat du script ci-dessus, puis cliquer sur Import. Les commandes interprétées s'affichent alors dans un tableau et il suffit de cliquer sur « Run » pour que les données soient finalement importées sur Wikidata.

La documentation de Quick Statements est disponible sur l'ancienne version de l'outil : http://tools.wmflabs.org/wikidata-todo/quick_statements.php